# Assignment in Pyspark



In [ ]:
##Creation of spark session
from pyspark.sql import SparkSession

In [60]:
spark=SparkSession.builder.appName('Assignment').getOrCreate()

In [61]:
import pandas as pd

In [62]:
# Raw Dataset of Australia Data
pf_AUS = pd.read_excel(r'C:\Users\Anupam Saha\Desktop\Incubyte DE Assessment\SampleInputData\AUS.xlsx')
pf_AUS

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination
0,1,Mike,LMN,NaT,2022-05-11 00:00:00
1,2,Jonnathan,XYZ,1997-12-13,2021-13-13
2,3,Cristina,ABC,1998-03-12,2022-03-12 00:00:00


In [131]:
pf_AUS['Date of Vaccination'] = pd.to_datetime(pf_AUS['Date of Vaccination'],errors ='coerce')
pf_AUS

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination
0,1,Mike,LMN,NaT,2022-05-11
1,2,Jonnathan,XYZ,1997-12-13,NaT
2,3,Cristina,ABC,1998-03-12,2022-03-12


In [130]:
#Converting the pandas Dataframe to Pyspark Dataframe
from pyspark.sql.types import *
spark.conf.set("spark.sql.execution.arrow.enabled","true")       
df=spark.createDataFrame(pf_AUS)

In [129]:
#Data Cleaning of Australia Dataset
from pyspark.sql.types import *
from pyspark.sql.functions import lit
spark.conf.set("spark.sql.execution.arrow.enabled","true")       
df=spark.createDataFrame(pf_AUS)
df_AUS =df.withColumn("Country", lit('AUS'))
df_AUS =df_AUS.withColumn("Free or Paid", lit('F'))
df_AUS = df_AUS.drop('Unique ID','Date of Birth','Date of Vaccination')
df_AUS=df_AUS.withColumnRenamed("Vaccine Type","VaccinationType")
df_AUS=df_AUS.withColumnRenamed("Patient Name","Name")
df_AUS.show()

+---------+---------------+-------+------------+
|     Name|VaccinationType|Country|Free or Paid|
+---------+---------------+-------+------------+
|     Mike|            LMN|    AUS|           F|
|Jonnathan|            XYZ|    AUS|           F|
| Cristina|            ABC|    AUS|           F|
+---------+---------------+-------+------------+



In [132]:
####Data Cleaning of India Dataset
from pyspark.sql.functions import lit
pf_IND=spark.read.csv(r'C:\Users\Anupam Saha\Desktop\Incubyte DE Assessment\SampleInputData\IND.csv',header=True,inferSchema=True)
df_IND=pf_IND.withColumn("Country", lit('IND'))
df_IND = df_IND.drop('ID','DOB','VaccinationDate')
df_IND = df_IND.select('Name','VaccinationType','Country','Free or Paid')
df_IND.show()

+------+---------------+-------+------------+
|  Name|VaccinationType|Country|Free or Paid|
+------+---------------+-------+------------+
| Vikas|            XYZ|    IND|           F|
| Rahul|            ABC|    IND|           P|
|Sameer|            ABC|    IND|           F|
+------+---------------+-------+------------+



In [133]:
#Raw Data of USA dataset
pf_USA = pd.read_csv(r'C:\Users\Anupam Saha\Desktop\Incubyte DE Assessment\SampleInputData\USA.csv')
pf_USA

,ID,Name,VaccinationType,VaccinationDate
0,1,Sam,EFG,6152022
1,2,John,XYZ,1052022
2,3,Mike,ABC,12282021


In [134]:
# Data filtering of the dataset
pf_USA['VaccinationDate'] = pd.to_datetime(pf_USA['VaccinationDate'],format='%m%d%Y')
pf_USA

,ID,Name,VaccinationType,VaccinationDate
0,1,Sam,EFG,2022-06-15
1,2,John,XYZ,2022-10-05
2,3,Mike,ABC,2021-12-28


In [70]:
#Converting the pandas dataframe to pyspark dataframe
from pyspark.sql.types import *
spark.conf.set("spark.sql.execution.arrow.enabled","true")       
pf1_USA=spark.createDataFrame(pf_USA)

In [135]:
#Data Cleaning of USA Dataset
from pyspark.sql.functions import lit
df_USA =pf1_USA.withColumn("Country", lit('USA'))
df_USA =df_USA.withColumn("Free or Paid", lit('F'))
df_USA = df_USA.drop('ID','VaccinationDate')
df_USA.show()

+----+---------------+-------+------------+
|Name|VaccinationType|Country|Free or Paid|
+----+---------------+-------+------------+
| Sam|            EFG|    USA|           F|
|John|            XYZ|    USA|           F|
|Mike|            ABC|    USA|           F|
+----+---------------+-------+------------+



In [137]:
##Data merging into single source of truth
df9 = df_AUS.union(df_IND)
df = df9.union(df_USA)
df.show()

+---------+---------------+-------+------------+
|     Name|VaccinationType|Country|Free or Paid|
+---------+---------------+-------+------------+
|     Mike|            LMN|    AUS|           F|
|Jonnathan|            XYZ|    AUS|           F|
| Cristina|            ABC|    AUS|           F|
|    Vikas|            XYZ|    IND|           F|
|    Rahul|            ABC|    IND|           P|
|   Sameer|            ABC|    IND|           F|
|      Sam|            EFG|    USA|           F|
|     John|            XYZ|    USA|           F|
|     Mike|            ABC|    USA|           F|
+---------+---------------+-------+------------+



In [138]:
####Total vaccination count by country and vaccination type (Metrix 1)
import pyspark.sql.functions as f
df11=df.groupBy('VaccinationType','Country').count().select('Country','VaccinationType',f.col('count').alias('No of Vaccinations')).orderBy('VaccinationType')
df11.show()

+-------+---------------+------------------+
|Country|VaccinationType|No of Vaccinations|
+-------+---------------+------------------+
|    AUS|            ABC|                 1|
|    USA|            ABC|                 1|
|    IND|            ABC|                 2|
|    USA|            EFG|                 1|
|    AUS|            LMN|                 1|
|    AUS|            XYZ|                 1|
|    USA|            XYZ|                 1|
|    IND|            XYZ|                 1|
+-------+---------------+------------------+



In [113]:
# Total Number of vaccination received per country
import pyspark.sql.functions as f
df12=df.groupBy('Country').count().select('Country',f.col('count').alias('No of Vaccination'))
df12.show()

+-------+-----------------+
|Country|No of Vaccination|
+-------+-----------------+
|    AUS|                3|
|    IND|                3|
|    USA|                3|
+-------+-----------------+



In [139]:
# Total population of the countries
pop = {'Country':['AUS','IND','USA'],'Population':[26,1393,329]}
dfg = pd.DataFrame(pop)
dfg1 = spark.createDataFrame(dfg)
joinedDF = df12.join(dfg1.select('Country','Population'), ['Country'])
joinedDF.show() 

+-------+-----------------+----------+
|Country|No of Vaccination|Population|
+-------+-----------------+----------+
|    AUS|                3|        26|
|    IND|                3|      1393|
|    USA|                3|       329|
+-------+-----------------+----------+



In [140]:
#% vaccination in each country(metrix 2)
import pyspark.sql.functions as f
joinedDF = joinedDF.withColumn(
    "% of vaccinated population", 
    f.col('No of Vaccination')/f.col('Population') *100
)
joinedDF = joinedDF.drop('No of Vaccination','Population')
joinedDF.show()

+-------+--------------------------+
|Country|% of vaccinated population|
+-------+--------------------------+
|    AUS|        11.538461538461538|
|    IND|       0.21536252692031585|
|    USA|         0.911854103343465|
+-------+--------------------------+



In [93]:
df.show()

+---------+---------------+-------+------------+
|     Name|VaccinationType|Country|Free or Paid|
+---------+---------------+-------+------------+
|     Mike|            LMN|    AUS|           F|
|Jonnathan|            XYZ|    AUS|           F|
| Cristina|            ABC|    AUS|           F|
|    Vikas|            XYZ|    IND|           F|
|    Rahul|            ABC|    IND|           P|
|   Sameer|            ABC|    IND|           F|
|      Sam|            EFG|    USA|           F|
|     John|            XYZ|    USA|           F|
|     Mike|            ABC|    USA|           F|
+---------+---------------+-------+------------+



In [141]:
#Total number of free vaccinated population
import pyspark.sql.functions as f
df4 =df.groupBy('Free or Paid','Country').count().select('Free or Paid','Country',f.col('count').alias('counts')).where(df['Free or Paid']=='F')
df4.show()

+------------+-------+------+
|Free or Paid|Country|counts|
+------------+-------+------+
|           F|    AUS|     3|
|           F|    IND|     2|
|           F|    USA|     3|
+------------+-------+------+



In [128]:
#% vaccination contribution by country(metrix 3) 
import pyspark.sql.functions as f
df4 = df4.withColumn(
    "% contribution", 
    f.col('counts')/3 *100
)
df4 = df4.drop('Free or Paid','counts')
df4.show()

+-------+-----------------+
|Country|   % contribution|
+-------+-----------------+
|    AUS|            100.0|
|    IND|66.66666666666666|
|    USA|            100.0|
+-------+-----------------+

